### Documentation for Google Earth Engine

Coding best practices
https://developers.google.com/earth-engine/guides/best_practices

API reference
https://developers.google.com/earth-engine/apidocs/

geemao library
https://geemap.org/foliumap/?h=addlayer#geemap.foliumap

geemap notebook examples
https://github.com/giswqs/geemap/tree/master/examples


## Precipitation + Evaporation layers -- NLDAS

Land Data Assimilation System (LDAS) combines multiple sources of observations (such as precipitation gauge data, satellite data, and radar precipitation measurements) to produce estimates of climatological properties at or near the Earth''s surface.

**This dataset is the primary (default) forcing file (File A) for Phase 2 of the North American Land Data Assimilation System (NLDAS-2). The data are in 1/8th-degree grid spacing; the temporal resolution is hourly.**




In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWhPhW6HVT682AjUudBGCr6LLdRuiazi-jHnw_CwvBoIG7qoH1l8oVM



Successfully saved authorization token.


In [6]:
i_year = 2010
years_back = 10

# --- Does not use data from i_year --
# Initial date of interest (inclusive)
i_date = str(i_year - years_back)  + '-01-01' 
# Final date of interest (exclusive)
f_date = str(i_year) + '-01-01'

#print(i_date, f_date)

2000-01-01 2010-01-01


In [28]:
nldas = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').filterDate(i_date,f_date)


evap = nldas.select('potential_evaporation')
print('evaporation filtered collection size:', evap.size().getInfo())

prec = nldas.select('total_precipitation')
print('precipitation filtered collection size:', prec.size().getInfo())

evaporation filtered collection size: 87672
precipitation filtered collection size: 87672


In [32]:
aridity_index = prec.sum().divide( evap.sum().multiply(10))

In [39]:
ca_bbox = ee.Geometry.Polygon([[[-124.409591,32], 
                               [-114.131211,32], 
                               [-114.131211,42.01], 
                               [-124.409591,42.01]]])

ca_aridity_index = aridity_index.clip(ca_bbox)

In [46]:
task = ee.batch.Export.image.toDrive(ca_aridity_index,
                     description= str(i_year)+'aridity_index'
                     )

task.start()

In [41]:
ca_aridity_index.projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [ ]:
max = ca_aridity_index.reduceRegion(reducer = ee.Reducer.max(),
                                 geometry = ca_bbox,
                                 scale=13915)
max.getInfo()

In [34]:
aridity_index.projection().getInfo()
#evap[1].projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [ ]:
ca_total_evap = evap_nldas.sum().clip(ca_bbox)
ca_total_prec = prec_nldas.sum().clip(ca_bbox)

total_evap = evap_nldas.sum()
total_prec = prec_nldas.sum()



#ai = total_prec/total_evap



In [ ]:
import geemap

nldas_map = geemap.Map()

vis_evap = { 'min':0, 'max':3*8760, 'palette': purples}
vis_prec = { 'min':0, 'max':125*8760, 'palette': purples}

nldas_map.addLayer(total_evap,vis_evap, "total evaporation")
nldas_map.addLayer(total_prec,vis_prec, "total precipitation")